In [ ]:
### NOTE:
### There are three different architecture used for WGAN in this notebook. Use the one you like.
### WGAN can be used used to produce RGB images. But In this notebook, we used it to produce grayscale images.
### Please provide path to folder/models where asked.
### It is recommended to use colab for this notebook.

# Importing libraries

In [ ]:
import os
import time
import numpy as np
import math
import sys
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch.autograd as autograd
import torch

# Initialization

In [ ]:
####### Initializing #######

batch_size = 2
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 8
latent_dim = 150
#latent_dim = 150
img_size = 64
#img_size = 224
channels = 1          ### It is recommended to keep it 1
n_critic = 5
clip_value = 0.01

img_shape = (channels, img_size, img_size)

cuda = True if torch.cuda.is_available() else False

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# **WGAN Architectures Definition**

## WGAN Architecture 1

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            nn.Linear(1024, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

## WGAN Architecture 2

In [ ]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        def block(in_feat, out_feat, normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(
            *block(latent_dim, 128, normalize=False),
            *block(128, 256),
            *block(256, 512),
            *block(512, 1024),
            *block(1024, 2048),
            *block(2048, 4096),
            nn.Linear(4096, int(np.prod(img_shape))),
            nn.Tanh()
        )

    def forward(self, z):
        img = self.model(z)
        img = img.view(img.shape[0], *img_shape)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
        )

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)
        validity = self.model(img_flat)
        return validity

## WGAN Architecture 3

In [ ]:
class GeneratorG(nn.Module):
    def __init__(self):
        super(GeneratorG, self).__init__()

        preprocess = nn.Sequential(
            nn.Linear(latent_dim, 2 * 2 * 768),
            nn.ReLU(True),
        )
        block1 = nn.Sequential( # 4
            nn.ConvTranspose2d(768, 512, 2, stride=2),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
        )
        block2 = nn.Sequential( #16
            nn.ConvTranspose2d(512, 256, 2, stride=2),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
        )

        block3 = nn.Sequential( #34
            nn.ConvTranspose2d(256, 128, 2, stride=2),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
        )
        block4 = nn.Sequential( #70
            nn.ConvTranspose2d(128, 64, 2, stride=2),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
        )
        deconv_out = nn.ConvTranspose2d(64, 1, 2, stride=2)

        self.preprocess = preprocess
        self.block1 = block1
        self.block2 = block2
        self.block3 = block3
        self.block4 = block4
        self.deconv_out = deconv_out
        self.tanh = nn.Tanh()

    def forward(self, input):
        output = self.preprocess(input)
        output = output.view(-1, 768, 2, 2)
        output = self.block1(output)
        output = self.block2(output)
        output = self.block3(output)
        output = self.block4(output)
        output = self.deconv_out(output)
        output = self.tanh(output)
        return output

class DiscriminatorD(nn.Module):
    def __init__(self):
        super(DiscriminatorD, self).__init__()
        main = nn.Sequential(
            nn.Conv2d(1, 64, 2, 2),
            nn.LeakyReLU(),
            nn.Conv2d(64, 128, 2, 2),
            nn.LeakyReLU(),
            nn.Conv2d(128, 256, 2, 2),
            nn.LeakyReLU(),
            nn.Conv2d(256, 512, 2, 2),
            nn.LeakyReLU(),
        )

        self.main = main
        self.linear = nn.Linear(8192, 1)

    def forward(self, input):
        output = self.main(input)
        output = output.view(-1, 8192)
        output = self.linear(output)
        return output

# Some Important functions definition

In [ ]:
def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
    alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)
    d_interpolates = D(interpolates)
    fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    # Get gradient w.r.t. interpolates
    gradients = autograd.grad(
        outputs=d_interpolates,
        inputs=interpolates,
        grad_outputs=fake,
        create_graph=True,
        retain_graph=True,
        only_inputs=True,
    )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

# Loading generator and discriminator

In [ ]:
#################### Load Generator and Discriminator ####################
generator = GeneratorG()
discriminator = DiscriminatorD()

#generator.load_state_dict(torch.load('*** Provide Path to saved Generator model ***'))
#discriminator.load_state_dict(torch.load('*** Provide Path to saved Discriminator model ***'))

<All keys matched successfully>

# Loading dataset and setting up optimizers

In [ ]:
# Loss weight for gradient penalty
lambda_gp = 10

if cuda:
    generator.cuda()
    discriminator.cuda()

# Configure data loader
data = datasets.ImageFolder('*** Provide Path to Target Datast ***', transform = transforms.Compose([transforms.ToTensor()]))
dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

# **Training**

In [ ]:
# ----------
#  Training
# ----------
start_time = time.time()
sample_interval = 5000
batches_done = 0
os.makedirs("gen_images", exist_ok=True)
n_epochs = 2500
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor


for epoch in range(n_epochs):
    for i, (imgs, _) in enumerate(dataloader):
        imgs = imgs.sum(1)/3
        imgs = imgs.view(imgs.shape[0],1,img_size,img_size)
        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0],  latent_dim))))

        # Generate a batch of images
        fake_imgs = generator(z)

        # Fake images
        fake_validity = discriminator(fake_imgs)
        # Real images
        real_validity = discriminator(real_imgs)

        # Gradient penalty
        gradient_penalty = compute_gradient_penalty(discriminator, real_imgs.data, fake_imgs.data)
        # Adversarial loss
        d_loss = -torch.mean(real_validity) + torch.mean(fake_validity) + lambda_gp * gradient_penalty

        d_loss.backward()
        optimizer_D.step()

        optimizer_G.zero_grad()

        # Train the generator every n_critic steps
        if i %  n_critic == 0:
            # -----------------
            #  Train Generator
            # -----------------

            # Generate a batch of images
            fake_imgs = generator(z)
            # Loss measures generator's ability to fool the discriminator
            # Train on fake images
            fake_validity = discriminator(fake_imgs)
            g_loss = -torch.mean(fake_validity)

            g_loss.backward()
            optimizer_G.step()

            if batches_done %  sample_interval == 0:
                save_image(fake_imgs.data, "gen_images/%d.png" %(int(batches_done/sample_interval)), nrow=2, normalize=True)
            batches_done +=  n_critic
    
    torch.save(generator.state_dict(), '*** Provide Path to save the Generator ***')
    torch.save(discriminator.state_dict(), '*** Provide Path to save the Discriminator ***')
    print("[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]" % (epoch+1,  n_epochs, i, len(dataloader), d_loss.item(), g_loss.item()) )
    print("\n\n--- %s Minutes ---" % ((time.time() - start_time)/60))
    print('')

[Epoch 100/2500] [Batch 391/392] [D loss: 0.150654] [G loss: -208.621277]
[Epoch 200/2500] [Batch 391/392] [D loss: -0.390665] [G loss: -544.014282]
[Epoch 300/2500] [Batch 391/392] [D loss: 3.136023] [G loss: -5.197406]
